In [1]:
from dataclasses import dataclass
from collections import defaultdict
import operator
from functools import reduce
from operator import itemgetter
from collections import defaultdict,Counter
import numpy as np
import re

In [2]:

@dataclass
class LabeledTextData:
    id_num: int
    label: str
    tokens: list

train = [LabeledTextData(42, 'cat',  "🐈 🐯 🐱 🐩 🐱".split()),
         LabeledTextData(43, 'dog',  "🐶 🐶 🐈 🐶 🐩 🐈 🐶 🐶".split()),
         LabeledTextData(45, 'cat',  "🐈 🐈 🐯 🐶 🐈".split()),
         LabeledTextData(45, 'cat',  "🐈 🐈 🐈".split()),
         LabeledTextData(48, 'dog',  "🐶 🐶 🐯 🐈 🐩 🐱 🐩 🐶 🐩 🐶 ".split()),
        ]

In [3]:

def doc_priors(labels):
    doc_priors = defaultdict(float)

    for label in labels:
        doc_priors[label] = sum(1 for d in labels if d == label) / len(labels)

    return doc_priors# Number of males


In [4]:
labels = [d.label for d in train]
prior = doc_priors(labels)
prior 

defaultdict(float, {'cat': 0.6, 'dog': 0.4})

In [5]:
def likelihood(train, labels):
    
    # A default dict of default dicts; inner default dict is probability
    cond_prob = defaultdict(lambda: defaultdict(float))

    for label in set(labels):
    
        label_tokens = []
        for i, doc in enumerate(train):
             # For a given label, get a list of all the tokens for all the docs 
            if labels[i] == label:
                label_tokens.extend(doc)

        for token in set(label_tokens):
            # Find conditional probability: token count / total count
            cond_prob[label][token] = label_tokens.count(token) / len(label_tokens) 

    return cond_prob

In [6]:
labels
train_x =[x.tokens for x in train]
cond_prob = likelihood(train_x, labels)
cond_prob

defaultdict(<function __main__.likelihood.<locals>.<lambda>()>,
            {'cat': defaultdict(float,
                         {'🐩': 0.07692307692307693,
                          '🐈': 0.5384615384615384,
                          '🐱': 0.15384615384615385,
                          '🐶': 0.07692307692307693,
                          '🐯': 0.15384615384615385}),
             'dog': defaultdict(float,
                         {'🐩': 0.2222222222222222,
                          '🐈': 0.16666666666666666,
                          '🐱': 0.05555555555555555,
                          '🐶': 0.5,
                          '🐯': 0.05555555555555555})})

In [7]:
cond_prob.keys()

dict_keys(['cat', 'dog'])

In [8]:
def product(iterable):
    return reduce(operator.mul, iterable, 1)

def post_prob(test, cond_prob, doc_priors):
    prob_predicted = defaultdict(float)
    for label in cond_prob.keys():
        # For each label, calculate the conditional probability based on the prior and the tokens that appear
        prob_predicted[label] = doc_priors[label] * product(cond_prob[label][t] for t in test)
    
    return prob_predicted


In [9]:
test = LabeledTextData(id_num=90, label=None, tokens="🐱".split())
# test = LabeledTextData(id_num=91, label=None, tokens="🐶 🐶".split()) 
# test = LabeledTextData(id_num=92, label=None, tokens="🐶 🐱".split())
# test = LabeledTextData(id_num=93, label=None, tokens="🐈 🐈 🐶 🐶 🐩 🐱 🐱".split())
# test = LabeledTextData(id_num=94, label=None, tokens="🐬 ".split()) # Out of sample prediction
test_x = test.tokens
prob_predicted = post_prob(test_x, cond_prob, prior)
prob_predicted

defaultdict(float, {'cat': 0.09230769230769231, 'dog': 0.022222222222222223})

In [10]:
label, prob = max(prob_predicted.items(),
                  key=itemgetter(1))
print("The predicted class is:", label)

The predicted class is: cat


In [11]:
class NaiveBayes_MN:
    def __init__(self):
        self.vocab = set()
        self.labelset = None
        self.n_items = None
        self.priors = defaultdict(float)
        self.prob_predicted = defaultdict(float)
        self.c_word_dict = defaultdict(lambda: defaultdict(float))
        self.class_count = defaultdict(float)
    
    def fit(self, x_train, labels):
        """Train the Naïve Bayes classifier using text data."""
        if not isinstance(x_train, list) or not all(isinstance(doc, list) for doc in x_train):
            raise ValueError("x_train must be a list of tokenized documents (list of lists).")
        
        self.labels = np.array(labels)
        self.n_items = len(labels)
        self.labelset = set(labels)

        word_dict = defaultdict(list)
        
        # Collect words for each label and build vocabulary
        for i, doc in enumerate(x_train):
            self.vocab.update(doc)  # Using a set to avoid duplicates
            word_dict[labels[i]].extend(doc)

        # Convert vocab to list for indexing
        self.vocab = list(self.vocab)
        self.vocab_len = len(self.vocab)

        # Count occurrences of words in each class
        for label in self.labelset:
            self.c_word_dict[label] = Counter(word_dict[label])
            self.class_count[label] = sum(self.c_word_dict[label].values())

        # Compute prior probabilities
        self.doc_priors()

    def doc_priors(self):
        """Calculate prior probabilities of each class."""
        for label in self.labelset:
            self.priors[label] = np.sum(self.labels == label) / self.n_items

    def likelihood(self, word, label):
        """Calculate log-likelihood of a word given a class label."""
        return np.log((self.c_word_dict[label][word] + 1.0) / 
                      (self.class_count[label] + self.vocab_len + 1.0))

    def post_prob(self, test):
        """Calculate posterior probabilities for a test document."""
        for label in self.labelset:
            self.prob_predicted[label] = np.log(self.priors[label]) 
            for word in test:
                self.prob_predicted[label] += self.likelihood(word, label)
        return self.prob_predicted

    def predict(self, test):
        """Predict labels for a given test dataset."""
        self.test_labels = []
        for doc in test:
            prob_predicted = self.post_prob(doc)
            label, _ = max(prob_predicted.items(), key=itemgetter(1))
            self.test_labels.append(label)
        return self.test_labels

In [12]:
clf = NaiveBayes_MN()
clf.fit(train_x, labels)

In [13]:
clf.class_count

defaultdict(float, {'cat': 13, 'dog': 18})

In [14]:
test_x

['🐱']

In [15]:

clf.predict([test_x])

['cat']

In [ ]:
from sklearn.datasets import fetch_20newsgroups
categories=['alt.atheism', 'soc.religion.christian','comp.graphics', 'sci.med'] 
newsgroups_train=fetch_20newsgroups(subset='train',categories=categories)

train_data=newsgroups_train.data #getting all trainign examples
train_labels=newsgroups_train.target #getting training labels

In [ ]:
def preprocess_string(str_arg):
    cleaned_str=re.sub('[^a-z\s]+',' ',str_arg,flags=re.IGNORECASE) #every char except alphabets is replaced
    cleaned_str=re.sub('(\s+)',' ',cleaned_str) #multiple spaces are replaced by single space
    cleaned_str=cleaned_str.lower() #converting the cleaned string to lower case
    
    return cleaned_str # returning the preprocessed string 

In [ ]:

train_data = [preprocess_string(x).split() for x in train_data]

In [ ]:
clf = NaiveBayes_MN()
clf.fit(train_data, train_labels)

In [ ]:
newsgroups_test=fetch_20newsgroups(subset='test',categories=categories) #loading test data
test_data=newsgroups_test.data #get test set examples
test_labels=newsgroups_test.target #get test set labels

In [ ]:
test_data = [preprocess_string(x).split() for x in test_data]

In [ ]:
predicted = clf.predict(test_data)

In [ ]:
print("Predicted accuracy is %.3f" %(np.sum(test_labels == predicted)/len(predicted)))
